## 1. Imports

In [1]:
import torch
from torchvision.transforms.functional import to_tensor
import segmentation_models_pytorch as smp
from typing import Final
import os
from torchvision import transforms as T
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
import rasterio
from torchvision.transforms import ToTensor, ToPILImage
from segmentation_models_pytorch.utils.metrics import IoU
from sklearn.metrics import precision_score, recall_score, f1_score
import statistics

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
import math, decimal
#! pip install dataframe_image
import scipy.stats as stats
import dataframe_image as dfi

d:\diploma\cv-corruption-research\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

## 2. Count metrics (in import_inference_models.ipynb:  0. Count and save metrics)

## 3. Load data

In [3]:
model_names = ['DeepLabV3+_efficientnet-b4' , 'MAnet_efficientnet-b4', 'Unet_mit-b2', 'Unet++_efficientnet-b5']
transf_names = ['adjust_contrast', 'adjust_gamma', 'adjust_hue', 'adjust_saturation', 'Grayscale','webp'
                ]

# параметры с наихудшими случаями
# pars = [{'contrast_factor':[0.2, 0.5, 0.8, 1.2, 1.5, 1.8, 2]}, {'gamma':[0.2, 0.5, 0.8, 1.2, 1.5, 1.8, 1.9, 2.3]}, {'hue_factor':[-0.5, -0.4, -0.2, -0.1, 0, 0.1, 0.2, 0.4, 0.5]}, 
#         {'saturation_factor':[0, 0.3, 0.5, 0.8, 1.3, 1.5, 1.8, 2]}, {'out_channels':[3]}, {'webp': [1]}]

# приближенные к реаьности
pars = [{'contrast_factor':[0.5, 0.8, 1.2, 1.5]}, {'gamma':[ 0.5, 0.8, 1.2, 1.5, 1.8, 1.9, 2.3]}, {'hue_factor':[-0.5, -0.4, -0.2, -0.1, 0, 0.1, 0.2, 0.4, 0.5]}, 
        {'saturation_factor':[0.5, 0.8, 1.3, 1.5]}, {'out_channels':[3]}, {'webp': [1]}]

metric_name = 'f1'

## 4. Count metrics

### 4.1 Distortion Error using the original image

$$
\displaystyle
mDErr_{d}^M
=
\left( \sum_{d=1}^n E_{d,p}^M \right) / \left( \sum_{k=1}^i E_{orig, k}^M \right)
$$

n - number of distortions  
i - number of images  
M - model  
d - distortion   
p - parametr of distortion

In [23]:
res_mDErr = []
for model_name in model_names: 
    
    orig = np.load(f'metrics_old/{model_name}_origin_{metric_name}.npy')
    Err_orig = np.sum(orig)
    sum_DErr = []
    for transf_name, par in zip(transf_names, pars):
        par_name, par_val_list = next(iter(par.items()))
        DErr = []
        for par_i in par_val_list: 
            Err = []
            distortion = np.load(f'metrics_old/{model_name}_{transf_name}_{par_i}_' + metric_name + '.npy')
            Err = list(distortion)
            DErr.append(np.sum(Err) / Err_orig)      

        sum_DErr += DErr
    
    mDErr =np.sum(sum_DErr) / len(sum_DErr)

    res_mDErr.append(mDErr)

df = pd.DataFrame(data = np.resize(res_mDErr, (1,4)) , columns=model_names)
df

,DeepLabV3+_efficientnet-b4,MAnet_efficientnet-b4,Unet_mit-b2,Unet++_efficientnet-b5
0,0.95969,0.960937,0.951551,0.931217


In [33]:
res_mDErr = []
for model_name in model_names: 
    
    orig = np.load(f'metrics_old/{model_name}_origin_{metric_name}.npy')
    Err_orig = np.sum(orig)
    sum_DErr = []
    for transf_name, par in zip(transf_names, pars):
        par_name, par_val_list = next(iter(par.items()))
        DErr = []
        for par_i in par_val_list: 
            Err = []
            distortion = np.load(f'metrics_old/{model_name}_{transf_name}_{par_i}_' + metric_name + '.npy')
            Err = list(distortion)
            DErr.append(np.sum(Err) / Err_orig)      

        sum_DErr += DErr
    
        mDErr = np.sum(sum_DErr) / len(sum_DErr)

        res_mDErr.append(mDErr)

df = pd.DataFrame(data = np.resize(res_mDErr, (4,4)) , columns=model_names, index=transf_names)
df

,DeepLabV3+_efficientnet-b4,MAnet_efficientnet-b4,Unet_mit-b2,Unet++_efficientnet-b5
adjust_contrast,0.964850,0.961153,0.953487,0.959690
adjust_gamma,0.981524,0.960841,0.952058,0.960937
adjust_hue,0.971689,0.946424,0.944907,0.951551
adjust_saturation,0.952301,0.913616,0.915250,0.931217


### 4.2 Distortion Error without  using the original image (along distortion axis)

$$
\displaystyle
DErr_{d,p}^M
=
\frac{ \sum_{k=1}^i E_{k,d,p}^M }{n} 
$$

$$
\displaystyle
sDErr^M
=
\sqrt{ \frac{ \sum_{k=1}^d \sum_{l=1}^p (DErr_{k,l}^M - \overline{DErr_{k,l}^M} )}{ n - 1 } } / \frac{ \sum_{k=1}^d \sum_{l=1}^p DErr_{k,l}^M }{n} 
$$

or

$$  
sDErr^M = std_{d}(mean_{p,i}(DErr)) / (mean_{d}((mean_{p,i}(DErr))))
$$  


n - number of distortions  
i - number of images  
M - model  
d - distortion   
p - parametr of distortion

In [34]:
res_sDErr = []
for model_name in model_names: 

    sum_DErr = []
    for transf_name, par in zip(transf_names, pars):
        par_name, par_val_list = next(iter(par.items()))
        DErr = []
        for par_i in par_val_list: 
            Err = []
            distortion = np.load(f'metrics_old/{model_name}_{transf_name}_{par_i}_' + metric_name + '.npy')
            Err = list(distortion)
            #DErr.append(statistics.harmonic_mean([x for x in Err if x != 0]))                  
            #DErr.append((len(Err) + 0.00001) / np.sum([1.0/x for x in Err if x != 0]))
            DErr.append(np.mean(Err)) 

        sum_DErr += DErr
    
    sDErr = np.std(sum_DErr) / np.mean(sum_DErr)

    res_sDErr.append(sDErr)

df = pd.DataFrame(data = np.resize(res_sDErr, (1,4)) , columns=model_names)
df

# sDErr =  [0.016884072867085872, 0.06427385943356195, 0.10539996544466741, 0.009147556316510517]   # harmonic_mean
# sDErr =  [0.02088167970346063, 0.02902086624954602, 0.03522840984359849, 0.03222201925334532]     # mean


,DeepLabV3+_efficientnet-b4,MAnet_efficientnet-b4,Unet_mit-b2,Unet++_efficientnet-b5
0,0.051355,0.055596,0.056781,0.079539


### 4.3 Distortion Error without using the original image (along image axis)


$$
\displaystyle
imgDErr_{i}^M
=
\sqrt{ \frac{ \sum_{k=1}^d \sum_{l=1}^p (DErr_{k,l}^M - \overline{DErr_{k,l}^M} )}{ n - 1 } } /
\frac{ \sum_{k=1}^d \sum_{l=1}^p E_{k,l}^M }{n\cdot p} 
$$  

$$  
\displaystyle
sDErr^M
= \frac{ \sum_{k=1}^i imgDErr_{k}^M }{i} 
$$  

or

$$  
sDErr^M = mean_i( std(DErr_{d,p}) / mean_{d,p}(DErr) )
$$  

n - number of distortions  
i - number of images  
M - model  
d - distortion   
p - parametr of distortion

In [32]:
model_names = ['DeepLabV3+_efficientnet-b4' , 'MAnet_efficientnet-b4', 'Unet_mit-b2', 'Unet++_efficientnet-b5']
transf_names = ['adjust_contrast', 'adjust_gamma', 'adjust_hue', 'adjust_saturation']

pars = [{'contrast_factor':[0.5, 0.8, 1.2, 1.5]}, {'gamma':[ 0.5, 0.8, 1.2, 1.5, 1.8, 1.9, 2.3]}, {'hue_factor':[-0.5, -0.4, -0.2, -0.1, 0, 0.1, 0.2, 0.4, 0.5]}, 
        {'saturation_factor':[0.5, 0.8, 1.3, 1.5]}]

metric_name = 'f1'

l = len(np.load(f'metrics_old/DeepLabV3+_efficientnet-b4_adjust_saturation_0.5_IoU.npy'))
epsilon = 1e-4

res_sDErr = []
for model_name in model_names: 
    model_DErr = []
    for img in range(l):
        img_DErr = []
        for transf_name, par in zip(transf_names, pars):
            par_name, par_val_list = next(iter(par.items()))
            DErr = []
            for par_i in par_val_list: 
                distortion = np.load(f'metrics_old/{model_name}_{transf_name}_{par_i}_' + metric_name + '.npy')
                Err = distortion[img]
                DErr.append(Err + epsilon) 

            std_DErr = np.std(DErr)
            mean_DErr = np.mean(DErr)
            
            img_DErr.append(std_DErr/mean_DErr)
    
        model_DErr.append(np.mean(img_DErr))

    res_sDErr.append(np.mean(model_DErr))

df = pd.DataFrame(data = np.resize(res_sDErr, (1,4)) , columns=model_names)
df


,DeepLabV3+_efficientnet-b4,MAnet_efficientnet-b4,Unet_mit-b2,Unet++_efficientnet-b5
0,0.292669,0.243345,0.159796,0.338095


In [30]:
np.load(f'metrics_old/Unet++_efficientnet-b5_adjust_saturation_1.5_f1.npy')

array([0.7918449 , 0.83149284, 0.73236636, 0.75654727, 0.81085532,
       0.81248509, 0.77185737, 0.77239047, 0.8066588 , 0.82730661,
       0.83760933, 0.79306871, 0.80727858, 0.85131953, 0.84390754,
       0.81784684, 0.259423  , 0.31482855, 0.13094362, 0.24785038,
       0.74973926, 0.12483395, 0.5483827 , 0.67795305, 0.29867486,
       0.69517013, 0.60934433, 0.77085372, 0.78781862, 0.76084147,
       0.76964574, 0.768241  , 0.74065753, 0.76581086, 0.        ,
       0.71603078, 0.76418001, 0.81599636, 0.82552889, 0.78907621,
       0.62916156, 0.85290674, 0.81711626, 0.81375667, 0.        ,
       0.62174512, 0.55028091, 0.2165853 , 0.57558242, 0.35046684,
       0.365762  , 0.01052081, 0.05043892, 0.54176059, 0.62286024,
       0.72406209, 0.56066867, 0.52848143, 0.22989988, 0.67762994,
       0.58644099, 0.11706311, 0.49321783, 0.        , 0.00438766,
       0.10184049, 0.        , 0.00370361, 0.        , 0.00938783,
       0.00384621, 0.61481112, 0.        , 0.        , 0.     